In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

BASE_URL = 'http://www.enron-mail.com/email/'

def get_content_from_url(url):
    """Fetches the content of a page by URL."""
    print(f"Fetching content from: {url}")
    response = requests.get(url)
    return response.text

def get_links_from_content(content):
    """Extracts all links from the page content."""
    print("Extracting links from content...")
    soup = BeautifulSoup(content, 'html.parser')
    links = [a['href'] for a in soup.find_all('a')
             if not a['href'].startswith('?')
             and a['href'] != "/email/"]
    return links[1:]

def parse_email(content, folder_path=""):
    """Parses the email content and extracts email information."""
    print("Parsing email content...")
    soup = BeautifulSoup(content, 'html.parser')
    email_dict = {'folder_path': folder_path}
    lines = soup.get_text().splitlines()

    for line in lines:
        if line.startswith(('From:', 'To:', 'Subject:', 'Cc:', 'Bcc:', 'Date:')):
            key, value = line.split(':', 1)
            email_dict[key.strip()] = value.strip()
        else:
            break

    return email_dict

def process_folder(url, folder_path="", visited=None):
    """Рекурсивно обрабатывает папки и электронные письма."""

    # Проверка на неверный URL
    if url.endswith('//'):
        print(f"Пропускаем неверный URL: {url}")
        return []

    print(f"Обработка папки: {url}")

    if visited is None:
        visited = set()

    if url in visited:
        return []

    visited.add(url)
    content = get_content_from_url(url)
    links = get_links_from_content(content)

    emails = []

    # Проверяем, является ли текущий URL ссылкой на письмо (не оканчивается на '/')
    if not url.endswith('/'):
        email_data = parse_email(content, folder_path)
        emails.append(email_data)
        return emails

    for link in links:
        full_url = BASE_URL + link
        if link.endswith('/'):
            new_folder_path = folder_path + "/" + link.rstrip('/')
            emails.append({'folder_path': new_folder_path})  # Сохраняем имя папки
            emails.extend(process_folder(full_url, new_folder_path, visited))
        else:
            email_content = get_content_from_url(full_url)
            email_data = parse_email(email_content, folder_path)
            emails.append(email_data)

    return emails

directories = [
    "allen-p/", "arnold-j/", "arora-h/", "badeer-r/", "bailey-s/", "bass-e/", "baughman-d/",
    "beck-s/", "benson-r/", "blair-l/", "brawner-s/", "buy-r/", "campbell-l/", "carson-m/",
    "cash-m/", "causholli-m/", "corman-s/", "crandell-s/", "cuilla-m/", "dasovich-j/", "davis-d/",
    "dean-c/", "delainey-d/", "derrick-j/", "dickson-s/", "donoho-l/", "donohoe-t/", "dorland-c/",
    "ermis-f/", "farmer-d/", "fischer-m/", "forney-j/", "fossum-d/", "gang-l/", "gay-r/", "geaccone-t/",
    "germany-c/", "gilbertsmith-d/", "giron-d/", "griffith-j/", "grigsby-m/", "guzman-m/", "haedicke-m/",
    "hain-m/", "harris-s/", "hayslett-r/", "heard-m/", "hendrickson-s/", "hernandez-j/", "hodge-j/",
    "holst-k/", "horton-s/", "hyatt-k/", "hyvl-d/", "jones-t/", "kaminski-v/", "kean-s/", "keavey-p/",
    "keiser-k/", "king-j/", "kitchen-l/", "kuykendall-t/", "lavorato-j/", "lay-k/", "lenhart-m/", "lewis-a/",
    "linder-e/", "lokay-m/", "lokey-t/", "love-p/", "lucci-p/", "maggi-m/", "mann-k/", "martin-t/",
    "may-l/", "mccarty-d/", "mcconnell-m/", "mckay-b/", "mckay-j/", "mclaughlin-e/", "merriss-s/", "meyers-a/",
    "mims-thurston-p/", "motley-m/", "neal-s/", "nemec-g/", "panus-s/", "parks-j/", "pereira-s/", "perlingiere-d/",
    "phanis-s/", "pimenov-v/", "platter-p/", "presto-k/", "quenet-j/", "quigley-d/", "rapp-b/", "reitmeyer-j/",
    "richey-c/", "ring-a/", "ring-r/", "rodrique-r/", "rogers-b/", "ruscitti-k/", "sager-e/", "saibi-e/",
    "salisbury-h/", "sanchez-m/", "sanders-r/", "scholtes-d/", "schoolcraft-d/", "schwieger-j/", "scott-s/",
    "semperger-c/", "shackleton-s/", "shankman-j/", "shapiro-r/", "shively-h/", "skilling-j/", "slinger-r/",
    "smith-m/", "solberg-g/", "south-s/", "staab-t/", "stclair-c/", "steffes-j/", "stepenovitch-j/", "stokley-c/",
    "storey-g/", "sturm-f/", "swerzbin-m/", "symes-k/", "taylor-m/", "tholt-j/", "thomas-p/", "townsend-j/",
    "tycholiz-b/", "ward-k/", "watson-k/", "weldon-c/", "whalley-g/", "whalley-l/", "white-s/", "whitt-m/",
    "williams-j/", "williams-w3/", "wolfe-j/", "ybarbo-p/", "zipper-a/", "zufferli-j/"
]
all_emails = []

for directory in directories:
    emails = process_folder(BASE_URL + directory)
    all_emails.extend(emails)

if all_emails:  # Используйте all_emails вместо emails
    keys = all_emails[0].keys()
    with open('emails.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(all_emails)  # Используйте all_emails вместо emails
else:
    print("No emails found!")

Обработка папки: http://www.enron-mail.com/email/allen-p/
Fetching content from: http://www.enron-mail.com/email/allen-p/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/all_documents/
Fetching content from: http://www.enron-mail.com/email/all_documents/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/contacts/
Fetching content from: http://www.enron-mail.com/email/contacts/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/deleted_items/
Fetching content from: http://www.enron-mail.com/email/deleted_items/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/discussion_threads/
Fetching content from: http://www.enron-mail.com/email/discussion_threads/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/inbox/
Fetching content from: http://www.enron-mail.com/email/inbox/
Extracting links from content...
Обработка папки: http://ww

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/tasks/
Fetching content from: http://www.enron-mail.com/email/tasks/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/universal_studios/
Fetching content from: http://www.enron-mail.com/email/universal_studios/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/vulcan_signs/
Fetching content from: http://www.enron-mail.com/email/vulcan_signs/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/weekly_report/
Fetching content from: http://www.enron-mail.com/email/weekly_report/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/arora-h/
Fetching content from: http://www.enron-mail.com/email/arora-h/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/deleted_items/
Fetching content from: http://www.enron-mail.com/email/deleted_items/
Extracting links fro

Parsing email content...
Обработка папки: http://www.enron-mail.com/email/beck-s/
Fetching content from: http://www.enron-mail.com/email/beck-s/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/_sent_mail/
Fetching content from: http://www.enron-mail.com/email/_sent_mail/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/aec/
Fetching content from: http://www.enron-mail.com/email/aec/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/all_documents/
Fetching content from: http://www.enron-mail.com/email/all_documents/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/analyst_assoc_program/
Fetching content from: http://www.enron-mail.com/email/analyst_assoc_program/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/apollo__beth/
Fetching content from: http://www.enron-mail.com/email/apollo__beth/
Extracting links from content...

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/hpl/
Fetching content from: http://www.enron-mail.com/email/hpl/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/hr/
Fetching content from: http://www.enron-mail.com/email/hr/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/inbox/
Fetching content from: http://www.enron-mail.com/email/inbox/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/inclement_weather_list/
Fetching content from: http://www.enron-mail.com/email/inclement_weather_list/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/india/
Fetching content from: http://www.enron-mail.com/email/india/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/it___misc_/
Fetching content from: http://www.enron-mail.com/email/it___misc_/
Extracting links from content...
Обработка папки: http://www

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/steel/
Fetching content from: http://www.enron-mail.com/email/steel/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/surveys/
Fetching content from: http://www.enron-mail.com/email/surveys/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/systems_problems/
Fetching content from: http://www.enron-mail.com/email/systems_problems/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/unfinaled_invoices/
Fetching content from: http://www.enron-mail.com/email/unfinaled_invoices/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/unify/
Fetching content from: http://www.enron-mail.com/email/unify/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/unread_mail/
Fetching content from: http://www.enron-mail.com/email/unread_mail/
Extracting links from content.

...

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/vinson___elkins/
Fetching content from: http://www.enron-mail.com/email/vinson___elkins/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/weather/
Fetching content from: http://www.enron-mail.com/email/weather/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/weldon-c/
Fetching content from: http://www.enron-mail.com/email/weldon-c/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/abraxas/
Fetching content from: http://www.enron-mail.com/email/abraxas/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/all_documents/
Fetching content from: http://www.enron-mail.com/email/all_documents/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/associate/
Fetching content from: http://www.enron-mail.com/email/associate/
Extracting links from content...
Обр

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/merchant_investments/
Fetching content from: http://www.enron-mail.com/email/merchant_investments/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/notes_inbox/
Fetching content from: http://www.enron-mail.com/email/notes_inbox/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/personal/
Fetching content from: http://www.enron-mail.com/email/personal/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/sent/
Fetching content from: http://www.enron-mail.com/email/sent/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/sent_items/
Fetching content from: http://www.enron-mail.com/email/sent_items/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/tasks/
Fetching content from: http://www.enron-mail.com/email/tasks/
Extracting links from content...
Обраб

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/all_documents/
Fetching content from: http://www.enron-mail.com/email/all_documents/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/commoditylogic/
Fetching content from: http://www.enron-mail.com/email/commoditylogic/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/deals/
Fetching content from: http://www.enron-mail.com/email/deals/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/deleted_items/
Fetching content from: http://www.enron-mail.com/email/deleted_items/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/discussion_threads/
Fetching content from: http://www.enron-mail.com/email/discussion_threads/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/inbox/
Fetching content from: http://www.enron-mail.com/email/inbox/
Extracting links f

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/tasks/
Fetching content from: http://www.enron-mail.com/email/tasks/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/ybarbo-p/
Fetching content from: http://www.enron-mail.com/email/ybarbo-p/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/all_documents/
Fetching content from: http://www.enron-mail.com/email/all_documents/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/cabot/
Fetching content from: http://www.enron-mail.com/email/cabot/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/calendar/
Fetching content from: http://www.enron-mail.com/email/calendar/
Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/contacts/
Fetching content from: http://www.enron-mail.com/email/contacts/
Extracting links from content...
Обработка папки: http://www

Extracting links from content...
Обработка папки: http://www.enron-mail.com/email/sent_items/
Fetching content from: http://www.enron-mail.com/email/sent_items/
Extracting links from content...


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

BASE_URL = 'http://www.enron-mail.com/email/'

def get_content_from_url(url):
    """Fetches the content of a page by URL."""
    print(f"Fetching content from: {url}")
    response = requests.get(url)
    return response.text

def get_links_from_content(content):
    """Extracts all links from the page content."""
    print("Extracting links from content...")
    soup = BeautifulSoup(content, 'html.parser')
    links = [a['href'] for a in soup.find_all('a')
             if not a['href'].startswith('?')
             and a['href'] != "/email/"]
    return links[1:]

def get_all_directories(base_url):
    """Извлекает все директории с главной страницы."""
    content = get_content_from_url(base_url)
    soup = BeautifulSoup(content, 'html.parser')
    directories = [a['href'] for a in soup.find_all('a') if a['href'].endswith('/') and not a['href'].startswith('?')]
    return directories

def parse_email(content, folder_path=""):
    """Parses the email content and extracts email information."""
    print("Parsing email content...")
    soup = BeautifulSoup(content, 'html.parser')
    email_dict = {'folder_path': folder_path}
    lines = soup.get_text().splitlines()

    for line in lines:
        if line.startswith(('From:', 'To:', 'Subject:', 'Cc:', 'Bcc:', 'Date:')):
            key, value = line.split(':', 1)
            email_dict[key.strip()] = value.strip()
        else:
            break

    return email_dict

def process_folder(url, folder_path="", visited=None):
    """Рекурсивно обрабатывает папки и электронные письма."""

    # Проверка на неверный URL
    if url.endswith('//'):
        print(f"Пропускаем неверный URL: {url}")
        return []

    print(f"Обработка папки: {url}")

    if visited is None:
        visited = set()

    if url in visited:
        return []

    visited.add(url)
    content = get_content_from_url(url)
    links = get_links_from_content(content)

    emails = []

    # Проверяем, является ли текущий URL ссылкой на письмо (не оканчивается на '/')
    if not url.endswith('/'):
        email_data = parse_email(content, folder_path)
        emails.append(email_data)
        return emails

    for link in links:
        full_url = BASE_URL + link
        if link.endswith('/'):
            new_folder_path = folder_path + "/" + link.rstrip('/')
            emails.append({'folder_path': new_folder_path})  # Сохраняем имя папки
            emails.extend(process_folder(full_url, new_folder_path, visited))
        else:
            email_content = get_content_from_url(full_url)
            email_data = parse_email(email_content, folder_path)
            emails.append(email_data)

    return emails

all_directories = get_all_directories(BASE_URL)

all_emails = []

for directory in all_directories:
    emails = process_folder(BASE_URL + directory)
    all_emails.extend(emails)

if all_emails:
    keys = all_emails[0].keys()
    with open('emails_1.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(all_emails)
else:
    print("No emails found!")

Fetching content from: http://www.enron-mail.com/email/


In [ ]:
import pandas as pd
df = pd.read_csv('emails.csv')
df['folder_path'] = df['folder_path'].str.replace('/', '', regex=False)
df

,folder_path
0,all_documents
1,contacts
2,deleted_items
3,discussion_threads
4,inbox
...,...
2562,calendar
2563,deleted_items
2564,discussion_threads
2565,inbox
